### Imports

In [1]:
from tqdm import tqdm
from collections import Counter
from nltk.corpus import stopwords
from itertools import combinations
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bert_embedding import BertEmbedding
from allennlp.commands.elmo import ElmoEmbedder

from transformers import *
import torch
import keras

import imp, gzip
import pickle, nltk
import gensim
import multiprocessing
from copy import deepcopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils as my_utils

Using TensorFlow backend.


### Definitions

In [2]:
def get_edges(i):
    t = np.where(i>0)[0]
    comb = combinations(t, 2)
    embeds = {j:[] for j in t}

    for p, q in comb:
        if word_similarity[p][q]:
            embeds[p] += [q]
            embeds[q] += [p]
    return embeds

In [3]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in tqdm(parse(path)):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [4]:
def process_df(df):
    df['text'] = my_utils.preprocess(df['text'])
    return df

In [5]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    for line in tqdm(f):
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

In [6]:
def get_edges_transformers(text):
    sentence = text.split(" ")

    if embedding_name == 'bert':
        results = bert_embedding(sentence)
        embed_vecs = np.array([i[1][0] for i in results])
    else:
        embed_vecs = elmo.embed_sentence(sentence)[2]

    l = np.array(list(set(sentence).intersection(words)))

    pp = np.array([i[1] for i in nltk.pos_tag(l)])
    pp[pp=='JJ'] = 1
    pp[pp=='JJR'] = 1
    pp[pp=='JJS'] = 1
    pp[pp=='NN'] = 1
    pp[pp=='NNS'] = 1
    pp[pp=='NNP'] = 1
    pp[pp=='NNPS'] = 1
    pp[pp!='1'] = 0
    pp = pp.astype(int)

    l = l[pp==1]

    word_embeddings = np.array([embed_vecs[sentence.index(i)] for i in l])

    word_similarity = cosine_similarity(word_embeddings)

    remove = np.where(word_similarity == 1)

    for i, j in zip(remove[0], remove[1]):
        word_similarity[i][j] = 0
        word_similarity[j][i] = 0

    word_similarity = word_similarity > cutoff
    word_similarity = word_similarity.astype(int)
    np.fill_diagonal(word_similarity, 0)

    inds = np.where(word_similarity==1)

    embeds = {words.index(j):[] for j in l}

    for i, j in zip(inds[0], inds[1]):
        embeds[words.index(l[i])] += [words.index(l[j])]

    return embeds

### Config

In [7]:
dataset_name = "twitter_airline_2"

min_df = 5
max_df = .5
max_features = 50000
cutoffs = [0.3, 0.6]

n_cores = 40
# n_docs = 50000

### Start

In [8]:
# dataset_ = getDF('datasets/reviews_Electronics_5.json.gz')
# dataset_.shape

In [9]:
dataset = pd.read_csv("datasets_raw/Tweets.csv")

In [10]:
dataset.shape

(14640, 15)

In [11]:
dataset = dataset[dataset.airline_sentiment_confidence>=0.5][['airline_sentiment', 'text']]

In [12]:
dataset.airline_sentiment = dataset.airline_sentiment.map({'negative':0, 'neutral':1, 'positive':2})

In [13]:
dataset = dataset.rename(columns={'text': 'text', 'airline_sentiment': 'sentiment'})

In [14]:
dataset.shape

(14404, 2)

In [15]:
n = int(dataset.shape[0]/n_cores)
list_df = [dataset[i:i+n] for i in range(0, dataset.shape[0],n)]

pool = multiprocessing.Pool(n_cores)
processed_list_df = pool.map(process_df, list_df)
pool.close()

dataset = pd.concat(processed_list_df)

In [16]:
dataset = dataset[dataset.text.apply(lambda x: len(x.split(" "))>6 and len(x.split(" "))<50)]

In [17]:
dataset.shape

(7168, 2)

In [18]:
n_docs = dataset.shape[0]

In [19]:
dataset.to_pickle("datasets/"+ dataset_name + "_" + str(n_docs) + "_dataset")

In [20]:
vectorizer = CountVectorizer(analyzer="word",tokenizer=None,preprocessor=None,
                             stop_words="english", max_features=max_features,
                             max_df=max_df, min_df=min_df)

In [21]:
wordOccurenceMatrix = vectorizer.fit_transform(dataset.text.tolist()).toarray()

In [22]:
barren = np.where(wordOccurenceMatrix.sum(1)==0)[0]

In [23]:
barren

array([], dtype=int64)

In [24]:
words = vectorizer.get_feature_names()

In [25]:
bertvocab = words + ['[CLS]', '[UNK]']

In [26]:
pd.DataFrame(bertvocab).to_csv("resources/bertvocab_" + dataset_name + "_" + str(n_docs)+".txt", header=None, index=None)

In [27]:
len(words)

1283

# Embeddings

### Glove

In [28]:
glove_embedding_dim = 300
glove_embeddings_index = loadGloveModel("nongit_resources/glove.6B.300d.txt")

1711it [00:00, 8530.43it/s]

Loading Glove Model


400000it [00:46, 8539.20it/s]

Done. 400000  words loaded!


In [29]:
len(words)

1283

In [30]:
glove_word_embeddings = []

for word in tqdm(words):
    emb = glove_embeddings_index.get(word, np.array([0]*glove_embedding_dim))
    glove_word_embeddings.append(emb.tolist())

glove_word_embeddings = np.array(glove_word_embeddings)

100%|██████████| 1283/1283 [00:00<00:00, 18127.80it/s]


In [31]:
g = ['glove', glove_word_embeddings]

### Fasttext

In [32]:
%%time
fasttext_embedding_dim = 300
fasttext_embeddings_index = gensim.models.KeyedVectors.load_word2vec_format("nongit_resources/wiki-news-300d-1M.vec")

CPU times: user 4min 28s, sys: 4.87 s, total: 4min 33s
Wall time: 4min 32s


In [33]:
fasttext_word_embeddings = []

for word in tqdm(words):
    emb = np.array([0]*glove_embedding_dim)
    try:
        emb = fasttext_embeddings_index[word]
    except:
        pass
    fasttext_word_embeddings.append(emb.tolist())

fasttext_word_embeddings = np.array(fasttext_word_embeddings)

100%|██████████| 1283/1283 [00:00<00:00, 11988.40it/s]


In [34]:
f = ['fasttext', fasttext_word_embeddings]

#### Grid

In [35]:
for embedding_name, word_embeddings in [g, f]:
    for cutoff in cutoffs:
        print(embedding_name, cutoff)
        word_similarity = cosine_similarity(word_embeddings)

        remove = np.where(word_similarity == 1)

        for i, j in zip(remove[0], remove[1]):
            word_similarity[i][j] = 0
            word_similarity[j][i] = 0

        word_similarity = word_similarity > cutoff
        word_similarity = word_similarity.astype(int)
        np.fill_diagonal(word_similarity, 0)

        wordOccuranceMatrixBinary = wordOccurenceMatrix.copy()
        wordOccuranceMatrixBinary[wordOccuranceMatrixBinary > 1] = 1

        pool = multiprocessing.Pool(n_cores)
        similar_words = pool.map(get_edges, wordOccuranceMatrixBinary)
        pool.close()
        pickle_out = open("resources/"+ dataset_name + "_" + str(n_docs) +"_" + embedding_name + "_" + str(cutoff) + ".pickle","wb")
        pickle.dump(similar_words, pickle_out)
        pickle_out.close()

glove 0.3
glove 0.6
fasttext 0.3
fasttext 0.6


## Bert Embedding & Attention

In [ ]:
embedding_name = 'bert'

In [ ]:
cutoff = 0.95

In [ ]:
pretrained_weights = 'bert-base-uncased'

In [ ]:
model = BertModel.from_pretrained(pretrained_weights, output_hidden_states=True, output_attentions=True)

In [ ]:
tokenizer = BertTokenizer(vocab_file="resources/bertvocab_" + dataset_name + ".txt", never_split=True, do_basic_tokenize=False)

In [ ]:
tokenized_text = [tokenizer.tokenize(i) for i in dataset.text]

In [ ]:
def get_tokenized_text(text):
    return [j for j in text if j in words]

In [ ]:
%%time
pool = multiprocessing.Pool(n_cores)
temp = pool.map(get_tokenized_text, tokenized_text)
pool.close()

In [ ]:
# %%time
# temp = []
# for i in tokenized_text:
#     t = [j for j in i if j in words]
#     temp.append(t)

In [ ]:
tokenized_text = temp

In [ ]:
indexed_tokens = [tokenizer.convert_tokens_to_ids(i) for i in tokenized_text]

In [ ]:
input_ids = keras.preprocessing.sequence.pad_sequences(indexed_tokens, padding='post', dtype='long', maxlen=max([len(i) for i in indexed_tokens]))

In [ ]:
input_ids = torch.tensor(input_ids)

In [ ]:
input_ids.shape

In [ ]:
input_ids = torch.split(input_ids, 500, dim=0)

In [ ]:
pad_length = [len(i) for i in indexed_tokens]

In [ ]:
idx = 0
similar_words_bert = []
similar_words_bert_attention = []

In [ ]:
for batch in tqdm(input_ids):

    all_embeddings, _, _, all_attentions = model(batch)
    idx_copy = deepcopy(idx)
    
    print(idx)
    for one_embedding in all_embeddings.detach().numpy():
        word_embeddings = one_embedding[:pad_length[idx]]
        word_similarity = cosine_similarity(word_embeddings)
        remove = np.where(word_similarity == 1.000) # to remove self words coupling

        for i, j in zip(remove[0], remove[1]):
            word_similarity[i][j] = 0
            word_similarity[j][i] = 0

        word_similarity = word_similarity > cutoff
        word_similarity = word_similarity.astype(int)
        np.fill_diagonal(word_similarity, 0)

        inds = np.where(word_similarity==1)
        embeds = {words.index(j):[] for j in tokenized_text[idx]}

        for i, j in zip(inds[0], inds[1]):
            embeds[words.index(tokenized_text[idx][i])] += [words.index(tokenized_text[idx][j])]
        similar_words_bert.append(embeds)
    
    idx = deepcopy(idx_copy)
    
    print(idx)
    
    for one_attentions in all_attentions[0].detach().numpy():

        one_side_edges = np.argmax(one_attentions[9], axis=1) #taking 9 layer of attention
        embeds = {words.index(j):[] for j in tokenized_text[idx]}

        for j, i in enumerate(one_side_edges[:pad_length[idx]]):
            if i < pad_length[idx]:
                embeds[words.index(tokenized_text[idx][i])] += [words.index(tokenized_text[idx][j])]
        similar_words_bert_attention.append(embeds)
        idx += 1
    print(idx)

In [ ]:
pickle_out = open("resources/"+ dataset_name + "_" + str(n_docs) +"_" + 'bert' + "_" + str(cutoff) + ".pickle","wb")
pickle.dump(similar_words_bert, pickle_out)
pickle_out.close()

In [ ]:
pickle_out = open("resources/"+ dataset_name + "_" + str(n_docs) +"_" + 'bert_attention'+ ".pickle","wb")
pickle.dump(similar_words_bert_attention, pickle_out)
pickle_out.close()

### Appendix

In [ ]:
# elmo = ElmoEmbedder()

In [ ]:
        ### POS
#         pp = np.array([i[1] for i in nltk.pos_tag(words)])
#         pp[pp=='JJ'] = 1
#         pp[pp=='JJR'] = 1
#         pp[pp=='JJS'] = 1
#         pp[pp=='NN'] = 1
#         pp[pp=='NNS'] = 1
#         pp[pp=='NNP'] = 1
#         pp[pp=='NNPS'] = 1
#         pp[pp!='1'] = 0
#         pp = pp.astype(int)

#         wordOccuranceMatrixBinary[:, np.where(pp!=1)[0]] = 0


In [ ]:
# wordOccuranceMatrixBinary[0].sum()

# np.sum(wordOccuranceMatrixBinary)

# Counter(np.array([i[1] for i in nltk.pos_tag(words)]))

# pp.sum()

# np.where(pp!=1)[0].shape

In [ ]:
# %%time
# for embedding_name in ['bert', 'elmo']:
#     for cutoff in cutoffs:
#         print(embedding_name, cutoff)
#         pool = multiprocessing.Pool(n_cores)
#         similar_words = pool.map(get_edges_transformers, dataset.text.tolist())
#         pool.close()
#         pickle_out = open("resources/"+ dataset_name + "_" + str(n_docs) + "_" + embedding_name + "_" + str(cutoff) + ".pickle","wb")
#         pickle.dump(similar_words, pickle_out)
#         pickle_out.close()

In [ ]:
#     pd = pd.apply(lambda x: convert_numbers(x))

In [ ]:
# def process_df(df):
#     df['text'] = preprocess(df['reviewText'])
    
# #     pool = multiprocessing.Pool(n_cores)
# #     df['cleaned'] = pool.map(process_l, df['text'].tolist())
# #     pool.close()
    
# #     df['text'] = df['cleaned'].apply(lambda x: " ".join(x))
#     return df

In [ ]:
# p = [item for sublist in dataset['cleaned'].tolist() for item in sublist]

In [ ]:
# sorted(Counter(p))

In [ ]:
# def process_l(s):
#     return [i.lemma_ for i in sp(s) if i.lemma_ not in '-PRON-']

In [ ]:
# l = dataset['text'].tolist()

In [ ]:
# pool = multiprocessing.Pool(n_cores)
# processed_l = pool.map(process_l, l)
# pool.close()

In [ ]:
# joblib.dump(sampler, "resources/sampler_20iter_0.5_1")

In [ ]:
# pickle_out = open("resources/amazon_muiscal_glove_0.4.pickle","wb")
# pickle.dump(similar_words, pickle_out)
# pickle_out.close()